In [36]:
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
from datasets import load_metric
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import re

from src.pruning_utils import prune_gpt2_layers, check_gpt_layer_sparsity

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
model = GPT2Model.from_pretrained('gpt2-xl')
#model = GPT2LMHeadModel.from_pretrained('gpt2')
#model.to(device)
#model.eval()
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [30]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2-xl')
set_seed(42)
generator("The man worked as a", max_length=10, num_return_sequences=5)

Downloading: 100%|██████████| 1.36M/1.36M [00:01<00:00, 1.28MB/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The man worked as a security guard in a factory'},
 {'generated_text': 'The man worked as a salesman in Israel and in'},
 {'generated_text': 'The man worked as a guard at the camp for'},
 {'generated_text': "The man worked as a cleaner for the store's"},
 {'generated_text': 'The man worked as a barman and was found'}]

In [34]:
def evaluate_glue(model, tokenizer):
    # Load model
    #     model = GPT2LMHeadModel.from_pretrained('gpt2')
    #model.to(device)
    model.eval()
    #     tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    # Load dataset
    dataset = load_dataset('glue', 'sst2')
    # Evaluate
    metric = load_metric('glue', 'sst2')
    def tokenize_function(examples):
        return tokenizer(examples['sentence'], padding='max_length', truncation=True)
    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    eval_dataloader = DataLoader(tokenized_datasets['test'], batch_size=8, shuffle=False)
    for batch in eval_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            metric.add_batch(predictions=predictions, references=labels)
    print(metric.compute())

In [37]:
evaluate_glue(model, tokenizer)

Found cached dataset glue (/Users/colevincentfrank/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 1256.78it/s]
<ipython-input-34-d18f4fe2723b>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', 'sst2')


ImportError: To be able to use glue, you need to install the following dependency: sklearn.
Please install it using 'pip install sklearn' for instance.

In [31]:
sum(p.numel() for p in model.parameters())

1557611200

In [28]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
# evaluate unaltered model

In [15]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [20]:
tokenizer.decode(output)

ValueError: invalid literal for int() with base 10: 'last_hidden_state'

In [7]:
from datasets import load_dataset
from evaluate import evaluator
from transformers import AutoModelForSequenceClassification, pipeline

data = load_dataset("imdb", split="test").shuffle(seed=42).select(range(1000))
task_evaluator = evaluator("text2text-generation")

Found cached dataset imdb (/Users/colevincentfrank/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
Loading cached shuffled indices for dataset at /Users/colevincentfrank/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-2eff9f118d84c6fe.arrow


KeyError: "Unknown task text2text-generation, available tasks are: dict_keys(['text-classification', 'image-classification', 'question-answering', 'token-classification'])."

In [9]:
model(torch.tensor(["hello there"]))

ValueError: too many dimensions 'str'

In [6]:
# evaluate
eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    data=data,
    label_mapping={"NEGATIVE": 0, "POSITIVE": 1},
    tokenizer=tokenizer
)

The model 'GPT2Model' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'EsmForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GPT2ForSequenceClassification', 'GPTNeoForSequenceClassification', 'GPTJForSequenceClassification', 'IBertForSequenceClassification', 'LayoutLMForSequenceClassification', 'LayoutLMv2ForSequenceCl

ImportError: To be able to use evaluate-metric/accuracy, you need to install the following dependencies['sklearn'] using 'pip install sklearn' for instance'

In [7]:
# sparsify at 10%, 50%, 90%, 95%, 99%

In [90]:
model.named_parameters

<bound method Module.named_parameters of GPT2Model(
  (wte): Embedding(50257, 1600)
  (wpe): Embedding(1024, 1600)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): GPT2Block(
      (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): GPT2Block(
      (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (

In [ ]:
# test

In [42]:
params = model.state_dict() 
#params.keys()

In [49]:
prune_gpt2_layers(0.9)